In [ ]:
## Production version of code for Insight Data Engineers program.

# Overall strategy - define module "account" which contains a list of "contacts"
# Contacts are saved as Python dictionary, using account number as key and the "tier" level as value
# "tier" being the shorthand for the number of degrees of separation.
# Since Python dictionaries are extremely fast (equivalent to hash tables), should be able to flag transaction fast

import pandas as pd
import csv

In [ ]:
## Paths to data files
input_dir = 'paymo_input/'
batch_file = 'batch_payment.csv'
stream_file = 'stream_payment.csv'
batch_path = input_dir + batch_file
stream_path = input_dir + stream_file

test_file = 'batch_test_2.csv'
test_path = input_dir + test_file

In [ ]:
## first hurdle - PayMo messages can include commas, and therefore using pandas csv_read fails due to inconsistent number of columns.
## current compromise is to use DictReader instead, which truncates Message at first comma
## best solution likely involves regular expressions, but not analyzing messages for the time being anyway

#takes about 30 seconds to load the 3 million lines each of batch_payment and stream_payment on my macbook pro.

batch_dict = {}
batch_dict['time'] = {}
batch_dict['id1'] = {}
batch_dict['id2'] = {}
batch_dict['amount'] = {}
batch_dict['message'] = {}

with open(batch_path) as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        batch_dict['time'][i] = row['time']
        batch_dict['id1'][i] = row[' id1']
        batch_dict['id2'][i] = row[' id2']
        batch_dict['amount'][i] = row[' amount']
        batch_dict['message'][i] = row[' message']
        
df_batch = pd.DataFrame.from_dict(batch_dict)

In [ ]:
## repeat the above - load in data from stream_payment and turn into pandas dataframe.
stream_dict = {}
stream_dict['time'] = {}
stream_dict['id1'] = {}
stream_dict['id2'] = {}
stream_dict['amount'] = {}
stream_dict['message'] = {}

with open(stream_path) as csvfile:
    reader = csv.DictReader(csvfile)
    for i, row in enumerate(reader):
        stream_dict['time'][i] = row['time']
        stream_dict['id1'][i] = row[' id1']
        stream_dict['id2'][i] = row[' id2']
        stream_dict['amount'][i] = row[' amount']
        stream_dict['message'][i] = row[' message']
        
df_stream = pd.DataFrame.from_dict(stream_dict)

In [ ]:
#dictionary columns are randomly ordered - reorder as expected
df_batch = df_batch[['time','id1','id2','amount','message']]
df_stream = df_stream[['time','id1','id2','amount','message']]